In [1]:
import os
import re
import pymorphy2
from math import log
from sklearn.feature_extraction.text import CountVectorizer
morph = pymorphy2.MorphAnalyzer()
def lemmatize(token):
    try: 
        gram_info = morph.parse(token)
        return gram_info[0].normal_form
    except:
        return token
def clean(text):
    # заменяем все знаки препинания, кроме дефиса, на пробелы
    text = re.sub('[!"#$%&\'()*+,\../:;<=>?@[\\]^_`{|}~«–»—№<>]+', ' ', text)
    words = text.lower().split()
    # лемматизируем, выкидывая всё, что является числом и чья длина 2 или меньше
    tokens = [lemmatize(w) for w in words if len(w) > 2 and not w.isdigit() and not w.split('-')[0].isdigit()]
    return tokens
def process_files(path):
    file2words = {}  # создаем пустой словарь
    for file in os.listdir(path):
        fileName = path + file
        with open(fileName, encoding='utf-8') as f:    # открываем;
            words = clean(f.read())  
            file2words[fileName] = words 
    return file2words
file2words = process_files('C:\\файлы1\\')
print ('файлы со словами',file2words)


файлы со словами {'C:\\файлы1\\раки.txt': ['кот', 'ехать', 'рак', 'хрома', 'собака'], 'C:\\файлы1\\кот.txt': ['они', 'ехать', 'кот', 'кот', 'задом', 'перед', 'ехать'], 'C:\\файлы1\\комарики.txt': ['они', 'лететь', 'комарик', 'воздушный', 'шарик'], 'C:\\файлы1\\медведи.txt': ['ехать', 'медведь', 'велосипед']}


In [2]:
clean_docs = [' '.join(words) for words in file2words.values()]
filenames = list(file2words.keys())
def index_one_file(words):
    fileIndex = {}                            
    for index, word in enumerate(words):   
        if word in fileIndex.keys():       
            fileIndex[word].append(index)  
        else:
            fileIndex[word] = [index]      
    return fileIndex

def make_index(file2words):
    index = {}
    for filename in file2words.keys():
        index[filename] = index_one_file(file2words[filename])
    return index
index = make_index(file2words)
print ('прямой индекс',index)


прямой индекс {'C:\\файлы1\\раки.txt': {'хрома': [3], 'ехать': [1], 'собака': [4], 'кот': [0], 'рак': [2]}, 'C:\\файлы1\\кот.txt': {'ехать': [1, 6], 'они': [0], 'перед': [5], 'задом': [4], 'кот': [2, 3]}, 'C:\\файлы1\\комарики.txt': {'шарик': [4], 'они': [0], 'лететь': [1], 'комарик': [2], 'воздушный': [3]}, 'C:\\файлы1\\медведи.txt': {'ехать': [0], 'медведь': [1], 'велосипед': [2]}}


In [4]:
def make_invertedIndex(index):
    invertedIndex, df = {}, {}
    
    for filename in index.keys():
        for word in index[filename].keys():
            if word in df.keys():
                df[word] += 1
            else:
                df[word] = 1 
            if word in invertedIndex.keys():
                if filename in invertedIndex[word].keys():
                    invertedIndex[word][filename].extend(index[filename][word][:])
                else:
                    invertedIndex[word][filename] = index[filename][word]
            else:
                invertedIndex[word] = {filename: index[filename][word]}
                
    N = len(filenames)
    idf = {word: log(N / df[word]) for word in df}    
    
    return invertedIndex,idf

invertedIndex, idf = make_invertedIndex(index)
print ('обратный индекс',invertedIndex)
print()
print ('IDF',idf)
import json
# у нас два словаря - invertedIndex, idf, чтобы не создавать два файла, сложим их все в один массив
data = [invertedIndex, idf, file2words]
with open('C:\\Users\Julia\index_idf_file2words.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False)


обратный индекс {'хрома': {'C:\\файлы1\\раки.txt': [3]}, 'воздушный': {'C:\\файлы1\\комарики.txt': [3]}, 'кот': {'C:\\файлы1\\раки.txt': [0], 'C:\\файлы1\\кот.txt': [2, 3]}, 'рак': {'C:\\файлы1\\раки.txt': [2]}, 'ехать': {'C:\\файлы1\\раки.txt': [1], 'C:\\файлы1\\кот.txt': [1, 6], 'C:\\файлы1\\медведи.txt': [0]}, 'собака': {'C:\\файлы1\\раки.txt': [4]}, 'они': {'C:\\файлы1\\кот.txt': [0], 'C:\\файлы1\\комарики.txt': [0]}, 'перед': {'C:\\файлы1\\кот.txt': [5]}, 'задом': {'C:\\файлы1\\кот.txt': [4]}, 'шарик': {'C:\\файлы1\\комарики.txt': [4]}, 'медведь': {'C:\\файлы1\\медведи.txt': [1]}, 'лететь': {'C:\\файлы1\\комарики.txt': [1]}, 'велосипед': {'C:\\файлы1\\медведи.txt': [2]}, 'комарик': {'C:\\файлы1\\комарики.txt': [2]}}

IDF {'хрома': 1.3862943611198906, 'кот': 0.6931471805599453, 'воздушный': 1.3862943611198906, 'рак': 1.3862943611198906, 'ехать': 0.28768207245178085, 'собака': 1.3862943611198906, 'они': 0.6931471805599453, 'перед': 1.3862943611198906, 'задом': 1.3862943611198906, 'ш